In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# # # use only for init
df= pd.DataFrame(columns=["f1_score","recall","precision","accuracy","confusion_matrix"])
output_file_name = "bgl_exp.csv"
output_exp_2 = "experiment_bgl_output_dry_run1.txt"
df.to_csv(output_file_name)


In [ ]:

import pickle
# output_file_name

In [ ]:
### Run statistical tests

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import torch
import torch.nn as nn
from tqdm import tqdm
from torchvision import transforms
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, WeightedRandomSampler

torch.random.seed = 0
np.random.seed(0)

In [ ]:
from datasets import *
from tokenizer import LogTokenizer
from utils import get_padded_data
from classes.loss_functions import LossCompute
from classes.model import TLogModel
from trainer import run_train, run_test


In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from tlog import TLogRunner
import pandas as pd


In [ ]:
def load_datasets(dataset_name):
    if dataset_name == "bgl":
        bgl_dataset = BGLDataset()
        every_n = 100
        aux_size = 100000
        spirit_dataset = SpiritDataset(every_n=every_n, aux_size=aux_size,aux=True)
        intrepid_dataset = InterpidScrubbedDataset(aux=True)
        max_lines = 5000000
        tbird_dataset = ThunderbirdDataset(max_lines=max_lines,aux=True)
        
        dataset = bgl_dataset
        aux_datasets = [tbird_dataset,spirit_dataset,intrepid_dataset]
        
    elif dataset_name == "tbird":
        bgl_dataset = BGLDataset(aux=True)
        every_n = 1
        aux_size = 100000
        spirit_dataset = SpiritDataset(every_n=every_n, aux_size=aux_size,aux=True)
        intrepid_dataset = InterpidScrubbedDataset(aux=True)
        max_lines = 5000000
        tbird_dataset = ThunderbirdDataset(max_lines=max_lines)
        
        dataset = tbird_dataset
        aux_datasets = [bgl_dataset,spirit_dataset,intrepid_dataset]
        
    elif dataset_name == "spirit":
        bgl_dataset = BGLDataset(aux=True)
        every_n = 100  # when spirit training use every_n=50
        aux_size = 100000
        spirit_dataset = SpiritDataset(every_n=every_n)
        intrepid_dataset = InterpidScrubbedDataset(aux=True)
        max_lines = 5000000
        tbird_dataset = ThunderbirdDataset(max_lines=max_lines,aux=True)
        
        dataset = spirit_dataset
        aux_datasets = [bgl_dataset,tbird_dataset,intrepid_dataset]
        
    elif dataset_name == "interpid":
        bgl_dataset = BGLDataset(aux=True)
        every_n = 100
        aux_size = 100000
        spirit_dataset = SpiritDataset(every_n=every_n,aux=True)
        intrepid_dataset = InterpidScrubbedDataset()
        max_lines = 5000000
        tbird_dataset = ThunderbirdDataset(max_lines=max_lines,aux=True)
        
        dataset = intrepid_dataset
        aux_datasets = [bgl_dataset,tbird_dataset,spirit_dataset]
    return dataset, aux_datasets

In [ ]:
experiment_params = []
# for dataset_name in ["bgl","tbird","spirit"]:
# for dataset_name in ["bgl", "spirit", "tbird"]:
# for dataset_name in ["bgl"]:
for dataset_name in ["bgl"]:
#     for split_size in [0.001, 0.05, 0.1, 0.2, 0.4, 0.6, 0.8]:
#     for split_size in [0.2, 0.4, 0.6, 0.8]:
    for split_size in [0.8]:
        for pretrain in [True]:
            for tokenizer in [True]:
                for use_generator in [True]:
                    experiment_params.append((dataset_name, pretrain, tokenizer, split_size, use_generator))

In [ ]:
experiment_params

In [ ]:
indecies_splits = {}

In [ ]:
fp = open(output_exp_2, 'a+')
for i, experiment in enumerate(experiment_params):
# for i, experiment in enumerate([('bgl', False, False, 0.2, False)]):
    print("Experiment:",i)
    print("Experiment:",i,file=fp)
    dataset_name,pretrain,tokenizer,split_size,use_generator = experiment
    print("---"*10)
    print("Running experiment with parameters:")
    print("Dataset:",dataset_name)
    print("Pretrain:",pretrain)
    print("Tokenizer:",tokenizer)
    print("Split size:",split_size)
    print("Generator:",use_generator)
    print("---"*10,file=fp)
    print("Running experiment with parameters:",file=fp)
    print("Dataset:",dataset_name,file=fp)
    print("Pretrain:",pretrain,file=fp)
    print("Tokenizer:",tokenizer,file=fp)
    print("Split size:",split_size,file=fp)
    print("Generator:",use_generator,file=fp)
    fp.flush()
    dataset, aux_datasets = load_datasets(dataset_name)

   
    if pretrain:
        model_weights_path= "./model_params_2classes_prototypes.pickle"
    else:
        model_weights_path = None

    if tokenizer:
        tokenizer_path = "./tokenizer_dict_model_params_model_params_2classes_prototypes.json"
    else:
        tokenizer_path = None


    model_params = {
        "tgt_vocab":16,
        "n_layers":2,
        "in_features":16,
        "out_features":16,
        "num_heads":2,
        "dropout":0.05,
        "max_len":50
    }

    
    if experiment[1] == False:
        batch_size=2048
        epochs=30
    else:
        if dataset_name=="spirit":
            if split_size == 0.001:
                batch_size=2048
            if split_size == 0.05:
                batch_size=1024
            if split_size == 0.2:
                batch_size=256
            if split_size == 0.4:
                batch_size=128
            if split_size == 0.6:
                batch_size=64
            if split_size == 0.8:
                batch_size=64
        batch_size=256
        epochs=3
    
    pad_len=50
    seed=0
    loss_criterion = nn.CrossEntropyLoss()#
   
    model_path='./models/'


    tlog_run = TLogRunner(dataset=dataset,
                       aux_datasets=aux_datasets,
                       model_params=model_params,
                       seed=seed,
                       split_size=split_size,
                       batch_size=batch_size,
                       pad_len=pad_len,
                       loss_criterion=loss_criterion,
                       model_path=model_path,
                       epochs=epochs,
                       model_weights_path=model_weights_path,
                       tokenizer_path=tokenizer_path,
                       use_generator=use_generator)

    load, labels = tlog_run.init_data(dataset, aux_datasets)
    log_payload, labels = tlog_run.tokenize_data(load,labels)
    
#     labels = np.where(labels==0, 1, 0)
    
    load_train, labels_train, load_test, labels_test = tlog_run.train_test_split(log_payload, labels.flatten())
    train_dataloader, test_dataloader = tlog_run.create_data_loaders(load_train, labels_train, load_test,
                                                                 labels_test)
    indecies_splits[str(split_size)] = train_dataloader.dataset.tensors[0].numpy().shape[0]
    torch.cuda.set_device(0)
    
    
    un, class_count = np.unique(labels_train, return_counts=True)
    
    print(class_count)
    calculate_weights = lambda x, i: x.sum() / (len(x)*x[i])
    weights = [calculate_weights(class_count,i) for i in range(len(class_count))]
    weights /= max(weights)

    class_weights=torch.FloatTensor(weights).cuda()
    
    if pretrain == False:
        class_weights=torch.FloatTensor(torch.tensor([0.3, 1.0])).cuda()
    
    tlog_run.loss_criterion = nn.CrossEntropyLoss(weight=class_weights)
    
    tlog_run.run_optimizer(train_dataloader, test_dataloader, log_payload, labels_test)
    max_f1=0
    best_res=None
    thresholds = []
    flag = True
    
    dataset_name = experiment[0]
    
    path = "./log_anomaly_detection_evaluation/" + dataset_name + "/"
    model_name = "tlog_"
    phase = "prediction_"
    with open (path + model_name + phase + "_".join([str(x) for x in experiment]) + ".pickle", "wb") as file:
        pickle.dump(tlog_run.max_distances, file)